In [124]:
#Dependencies
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [11]:
#Initialize browser
#Change chromedriver path when uploading to server

executable_path = {'executable_path': r"C:\Users\slapp\Desktop\Misc_class\chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)


In [12]:
#define initial url and direct browser
url = 'https://www.indeed.com/jobs?q=Data+Scientist&explvl=entry_level&fromage=3'
browser.visit(url)

In [7]:
#Make some soup
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [13]:
#Scrap all postings on page into a list. Run through loop to click on next page
postings = soup.find_all( class_= 'jobsearch-SerpJobCard')

In [16]:
postings[0]

<div class="jobsearch-SerpJobCard unifiedRow row result clickcard" data-ci="350551151" data-empn="4014643344856247" data-jk="73c01b3b82193108" id="pj_73c01b3b82193108">
<style>
.jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}
</style>
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0De9TWE4UUmup4W_MxodPnJDMB17E76eLaHvHXIOZ-Jlb186R68ROXsMo7aDXDfsoVEgE-upeW6I6w4r4b2Xc3oXlH5pykDNMkhvqC2_YL_JcOnhF97E6nEOSl4jGVnAho5ApBF6P2oLMV9QqFpkTjOBU-351Rugt7LLBzC4x_D92qNRmi9yL2Grh6TjSdddzNv1MhrTekGIwA9e4wu1rAICCg-INcpB6gjwgVqd61jzwt75XVSdbKsJqaRo7K-5sd3TNrkex3IQd_BJQOKgoTX-oGxKdWYKNTbDRb7_YQntjOWh7FRV-un_iJB_wbdRxAfChYvflTohSnz_haoP6u5lQgoEsy122ARP6n9W0UAKj3AAABsiaaB3hJmbx9vkKtRdbcnZcsKh7i9UX67HJ_wIwFuAVKJyOyoXuoK6e9UWWljQgMP6x0Oyx16j1qZDzUZafPEu8ETrQ==&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefineByCookie(['explvl']); sjoc('sja0', 1); convCtr('SJ'); rclk(this,jobmap[0],true,1);" on

In [79]:
len(postings)

17

In [37]:
#Scrape url and add domaine
f"https://www.indeed.com{postings[0].find('h2').find(class_ = 'jobtitle')['href']}"


'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0De9TWE4UUmup4W_MxodPnJDMB17E76eLaHvHXIOZ-Jlb186R68ROXsMo7aDXDfsoVEgE-upeW6I6w4r4b2Xc3oXlH5pykDNMkhvqC2_YL_JcOnhF97E6nEOSl4jGVnAho5ApBF6P2oLMV9QqFpkTjOBU-351Rugt7LLBzC4x_D92qNRmi9yL2Grh6TjSdddzNv1MhrTekGIwA9e4wu1rAICCg-INcpB6gjwgVqd61jzwt75XVSdbKsJqaRo7K-5sd3TNrkex3IQd_BJQOKgoTX-oGxKdWYKNTbDRb7_YQntjOWh7FRV-un_iJB_wbdRxAfChYvflTohSnz_haoP6u5lQgoEsy122ARP6n9W0UAKj3AAABsiaaB3hJmbx9vkKtRdbcnZcsKh7i9UX67HJ_wIwFuAVKJyOyoXuoK6e9UWWljQgMP6x0Oyx16j1qZDzUZafPEu8ETrQ==&p=0&fvj=1&vjs=3'

In [38]:
#Scrape job title
postings[0].find('h2').find(class_ = 'jobtitle').text.replace('\n', '')


'Backend Developer/Data Scientist'

In [39]:
#scrape company name
postings[0].find(class_ = 'company').text.replace('\n', '')

'Double Chain, Inc'

In [91]:
#scrape location (split into city and state later)
postings[0].find(class_ = 'recJobLoc')['data-rc-loc']

'Chino, CA'

In [ ]:
#splitting location into city and state. try except to include remote work
cities = []
states = []
for x in range(0, len(location)):
    citystate = location[x].split(', ')
    cities.append(citystate[0])
    try:
        states.append(citystate[1])
    except IndexError:
        states.append(citystate[0])
    #print(cities[x] + states[x])

In [41]:
#scrape date added (current day - "days posted ago")
postings[0].find(class_='date').text

'4 days ago'

In [121]:
#Convert result to int
testdate = int(postings[0].find(class_='date').text.split(' ')[0])

In [126]:
#subtract from current date w/ date time
d = datetime.today() - timedelta(days=testdate)
d

datetime.datetime(2020, 6, 18, 16, 12, 20, 979278)

In [46]:
#scrape salary, add null if not included
salaryrange = []
try:
    salaryrange.append(postings[1].find(class_ = 'salaryText').text.replace('\n', ''))
except AttributeError:
    salaryrange.append('NaN')
salaryrange

['NaN']

In [97]:
#scrape job abstract
postings[0].find(class_ = 'summary').find_all('li')

[<li style="margin-bottom:0px;">Experience with complex <b>data</b> problems and an interest in machine learning and artificial intelligence.</li>,
 <li>Constantly look for improvements in tools, <b>data</b> sources,…</li>]

In [59]:
postings[0].find(class_ = 'summary').find_all('li')

[<li style="margin-bottom:0px;">Experience with complex <b>data</b> problems and an interest in machine learning and artificial intelligence.</li>,
 <li>Constantly look for improvements in tools, <b>data</b> sources,…</li>]

In [129]:
#Collect all above data for each entry with a loop
post_url = []
job_title = []
company = [] 
city = []
state = []
date_added = []
salaryrange = []

        
for x in range(0, len(postings)):
    post_url.append(f"https://www.indeed.com{postings[x].find('h2').find(class_ = 'jobtitle')['href']}")
    job_title.append(postings[x].find('h2').find(class_ = 'jobtitle').text.replace('\n', ''))
    company.append(postings[x].find(class_ = 'company').text.replace('\n', ''))
    location = postings[x].find(class_ = 'recJobLoc')['data-rc-loc'].split(', ')
    city.append(location[0])
    try:
        state.append(location[1])
    except IndexError:
        state.append(location[0])
    temp_date = int(postings[0].find(class_='date').text.split(' ')[0])
    date_added.append(datetime.today() - timedelta(days=testdate))
    try:
        salaryrange.append(postings[x].find(class_ = 'salaryText').text.replace('\n', ''))
    except AttributeError:
        salaryrange.append('NaN')

In [131]:
state

['CA',
 'VA',
 'CA',
 'TX',
 'CO',
 'TX',
 'TX',
 'Remote',
 'TX',
 'Remote',
 'Remote',
 'TX',
 'CA',
 'NJ',
 'CA',
 'VA',
 'TX']

In [ ]:
#loop through pages